# BGT zadanie 5
## Autors
- Paweł Benkowski
- Stanisław Lutkiewicz"
### Task
Downloads Nietzsche and freud books count group of words in it(tiny,small,medium,big).  
Match language by analyzing character frequency  
Covid cases in India region   

In [1]:
import sys
!{sys.executable} -m pip install PyPDF2
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install bs4
!{sys.executable} -m pip install nltk

In [2]:
from pyspark import SparkContext, SparkFiles
from pathlib import Path
from bs4 import BeautifulSoup
from collections import Counter
from operator import add
import os
import requests
import PyPDF2
import re
import nltk
import timeit
nltk.download("averaged_perceptron_tagger")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
def get_books_urls(url):
    urlsWithName = []
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    
    for a in soup.find_all("a", href=True):
        arr = a["href"].split(".")
        if arr[-1] == "pdf":
            urlsWithName.append(("http:" +a["href"],a.string))
    return urlsWithName

In [4]:
def get_books_and_create_folder(urls, target):
    if not os.path.isdir(target):
        os.makedirs(target)
    for url in urls:
        file = Path(target,url[1] + ".pdf")
        if not file.exists():
            response = requests.get(url[0])
            with open(Path(target, url[1] + ".pdf"), "wb+") as f:
                f.write(response.content)
        else:
            print("File already exists")


In [5]:
def save_text_from_pdf(name):
    file = Path(name + ".txt")
    if not file.exists():
        fileReader = PyPDF2.PdfFileReader(name)
        with open(Path(name + ".txt"), "a+") as f:
            for page in fileReader.pages:
                f.write(page.extractText())

In [6]:
def is_noun(word : str) -> bool:
    if "s" == word or "i" == word:
        return False
    partOfSpeech = nltk.pos_tag([word])
    if "NN" == partOfSpeech[0][1]:
        return True
    return False

In [7]:
def compose_text_files(textFiles):
    if len(textFiles) == 1 :
        return textFiles[0].name
    with open(textFiles[0],'a') as toAppend:
        for txt in textFiles[1::]:
            with open(txt) as fromAppend:
                toAppend.write(fromAppend.read())
            os.remove(txt)
    return textFiles[0].name

In [8]:
sc = SparkContext("local", "first app")

In [10]:
folderNamesWithUrls = [('freud',"https://holybooks.com/sigmund-freud-the-complete-works/"),\
    ('nitsche',"https://holybooks.com/the-complete-works-of-friedrich-nietzsche-in-english-as-pdf/")]
files = []
for fnwu in folderNamesWithUrls:
    urls = get_books_urls(fnwu[1])
    get_books_and_create_folder(urls, fnwu[0])
    pdfFiles = list(Path(fnwu[0]).glob('*.pdf'))
    for pdf in pdfFiles:
        save_text_from_pdf(os.path.join(fnwu[0], pdf.name))
    textFileName = compose_text_files(list(Path(fnwu[0]).glob('*.txt')))
    files.append(sc.textFile(os.path.join(fnwu[0], textFileName)).cache())

File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists


In [ ]:
def count_letters_and_print_ratio(lettersRdd):
    lettersForCounting = lowerCaseLetters.map(lambda x: (x,1))
    countedLetters =  lettersForCounting.reduceByKey(add).collect()
    kLetters = Counter(dict(countedLetters))
    mostCommonLetters = kLetters.most_common(3)
    allLetters = lowerCaseLetters.count()
    print(f"top letters {mostCommonLetters}")
    print(f"total letters {allLetters}")
    for mcl in mostCommonLetters:
        print(f"{mcl[0]} : {(mcl[1]/allLetters)*100}%")

In [34]:
start = timeit.default_timer()
for file in files: # paralize -> foreach
    startIn = timeit.default_timer()
    print(file.name)
    lowerCaseText = file.map(lambda x: x.lower())
    lowerCaseLetters = lowerCaseText.flatMap(lambda x: re.findall(r"[a-zA-Z]", x))
    count_letters_and_print_ratio(lowerCaseLetters)

    words = lowerCaseText.flatMap(lambda x : re.findall(r"\b[^\d\W]+\b", x))
   
    wordsForCounting = words.map(lambda x: (x,1))
    countedNouns = wordsForCounting.reduceByKey(add).filter(lambda x: is_noun(x[0])).collect()
    
    kNouns = Counter(dict(countedNouns))
    kWords = Counter(words.collect())
    
    print(f"total word {words.count()}")
    print(f"most common {kWords.most_common(10)}")
    print(f"top nouns {kNouns.most_common(3)}")
    print(f"time spend {timeit.default_timer() - startIn}")
    print(30*"----")
print(f"total time spend {timeit.default_timer()-start}")

<bound method RDD.name of freud/Sigmund Freud – The Complete Works.pdf.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0>
['f', 'r', 'e', 'u', 'd', 'c', 'o', 'm', 'p', 'l']
top letters [('e', 1204065), ('t', 931463), ('i', 759718)]
total letters 9572627
e : 12.578208677722428%
t : 9.730484641258872%
i : 7.936358535645439%
total word 2068906
most common [('the', 142371), ('of', 94924), ('to', 60988), ('in', 50950), ('a', 48808), ('and', 46208), ('that', 33805), ('is', 30431), ('it', 27788), ('which', 22649)]
top nouns [('dream', 6115), ('analysis', 3584), ('time', 3456)]
time spend 37.55681641600677
------------------------------------------------------------------------------------------------------------------------
<bound method RDD.name of nitsche/The complete works of Friedrich Nietzsche VOL XVI.pdf.txt MapPartitionsRDD[3] at textFile at NativeMethodAccessorImpl.java:0>
['t', 'h', 'e', 'c', 'o', 'm', 'p', 'l', 'e', 't']
top letters [('e', 1464448), ('t', 112498